In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
# Function to calculate age
def calculate_age(birthdate):
    if pd.isna(birthdate):
        return None
    today = datetime.today()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

# Function to extract numeric value from string
def extract_numeric(value):
    numeric_string = re.sub(r'[^\d.]', '', value)
    return float(numeric_string) if numeric_string else 0.0

def lowercase(text):
    # Convert to lowercase
    return text.lower()

In [3]:
# Dictionary to store the DataFrames
dataframes = {}

# List of filenames
files = ['Customers', 'Exchange_Rates', 'Products', 'Sales', 'Stores']

# Try different encodings for each file
for file in files:
    for encoding in ['utf-8', 'latin1', 'iso-8859-1']:
        try:
            df = pd.read_csv(f"Dataset-Copy1/{file}.csv", encoding=encoding)
            dataframes[file] = df
            break
        except UnicodeDecodeError:
            continue
    else:
        print(f"Failed to load {file} with utf-8, latin1, or iso-8859-1 encodings")

In [4]:
x= list(dataframes.keys())
for i in x:
    dataframes[i].columns = dataframes[i].columns.str.lower().str.replace(" ", "_")
df_customer,df_product,df_sales,df_stores,df_exchange_rates = dataframes[x[0]],dataframes[x[2]],dataframes[x[3]],dataframes[x[4]],dataframes[x[1]]

In [5]:
for i in df_customer.columns:
    if df_customer[i].dtype == "object" and i not in ['state_code','zip_code','birthday'] :
        df_customer[i] = df_customer[i].apply(lowercase)
    print(f"{i}: count -> {df_customer[i].nunique()}, percentage ->{(df_customer[i].nunique()/len(df_customer))*100}%, null percentage ->{(df_customer[i].isnull().sum()/len(df_customer))*100}%")
print(df_customer.info())
df_customer.head()

customerkey: count -> 15266, percentage ->100.0%, null percentage ->0.0%
gender: count -> 2, percentage ->0.013101008777675883%, null percentage ->0.0%
name: count -> 15118, percentage ->99.03052535045198%, null percentage ->0.0%
city: count -> 8258, percentage ->54.094065243023714%, null percentage ->0.0%
state_code: count -> 467, percentage ->3.0590855495873184%, null percentage ->0.0655050438883794%
state: count -> 512, percentage ->3.353858247085026%, null percentage ->0.0%
zip_code: count -> 9505, percentage ->62.262544215904626%, null percentage ->0.0%
country: count -> 8, percentage ->0.05240403511070353%, null percentage ->0.0%
continent: count -> 3, percentage ->0.019651513166513822%, null percentage ->0.0%
birthday: count -> 11270, percentage ->73.82418446220359%, null percentage ->0.0%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 

,customerkey,gender,name,city,state_code,state,zip_code,country,continent,birthday
0,301,female,lilly harding,wandearah east,SA,south australia,5523,australia,australia,7/3/1939
1,325,female,madison hull,mount budd,WA,western australia,6522,australia,australia,9/27/1979
2,554,female,claire ferres,winjallok,VIC,victoria,3380,australia,australia,5/26/1947
3,786,male,jai poltpalingada,middle river,SA,south australia,5223,australia,australia,9/17/1957
4,1042,male,aidan pankhurst,tawonga south,VIC,victoria,3698,australia,australia,11/19/1965


In [6]:
# As per above data for Customers table, need to change dtypes for Birthday and Zip Code column as datetime and numeric
df_customer['birthday'] = pd.to_datetime(df_customer['birthday'], errors='coerce')
df_customer['age'] = df_customer['birthday'].apply(calculate_age)
df_customer['state_code'] = df_customer['state_code'].fillna(df_customer['state'])

In [7]:
for i in df_product.columns:
    if df_product[i].dtype == "object" and i not in ['unit_cost_usd','unit_price_usd']:
        df_product[i] = df_product[i].apply(lowercase)
    print(f"{i}: count -> {df_product[i].nunique()}, percentage ->{(df_product[i].nunique()/len(df_product))*100}%, null percentage ->{(df_product[i].isnull().sum()/len(df_product))*100}%")
print(df_product.info())
df_product.head()

productkey: count -> 2517, percentage ->100.0%, null percentage ->0.0%
product_name: count -> 2517, percentage ->100.0%, null percentage ->0.0%
brand: count -> 11, percentage ->0.43702820818434646%, null percentage ->0.0%
color: count -> 16, percentage ->0.6356773937226857%, null percentage ->0.0%
unit_cost_usd: count -> 480, percentage ->19.07032181168057%, null percentage ->0.0%
unit_price_usd: count -> 426, percentage ->16.924910607866508%, null percentage ->0.0%
subcategorykey: count -> 32, percentage ->1.2713547874453714%, null percentage ->0.0%
subcategory: count -> 32, percentage ->1.2713547874453714%, null percentage ->0.0%
categorykey: count -> 8, percentage ->0.31783869686134286%, null percentage ->0.0%
category: count -> 8, percentage ->0.31783869686134286%, null percentage ->0.0%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0

,productkey,product_name,brand,color,unit_cost_usd,unit_price_usd,subcategorykey,subcategory,categorykey,category
0,1,contoso 512mb mp3 player e51 silver,contoso,silver,$6.62,$12.99,101,mp4&mp3,1,audio
1,2,contoso 512mb mp3 player e51 blue,contoso,blue,$6.62,$12.99,101,mp4&mp3,1,audio
2,3,contoso 1g mp3 player e100 white,contoso,white,$7.40,$14.52,101,mp4&mp3,1,audio
3,4,contoso 2g mp3 player e200 silver,contoso,silver,$11.00,$21.57,101,mp4&mp3,1,audio
4,5,contoso 2g mp3 player e200 red,contoso,red,$11.00,$21.57,101,mp4&mp3,1,audio


In [8]:
for i in df_product.columns:
    if df_product[i].nunique() < 500:
        print(f"{i}:{df_product[i].unique()}")

brand:['contoso' 'wide world importers' 'northwind traders' 'adventure works'
 'southridge video' 'litware' 'fabrikam' 'proseware' 'a. datum'
 'the phone company' 'tailspin toys']
color:['silver' 'blue' 'white' 'red' 'black' 'green' 'orange' 'pink' 'yellow'
 'purple' 'brown' 'grey' 'gold' 'azure' 'silver grey' 'transparent']
unit_cost_usd:['$6.62 ' '$7.40 ' '$11.00 ' '$30.58 ' '$35.72 ' '$50.56 ' '$61.62 '
 '$91.93 ' '$84.49 ' '$48.92 ' '$99.14 ' '$106.69 ' '$76.45 ' '$91.95 '
 '$98.07 ' '$79.53 ' '$83.24 ' '$13.10 ' '$22.05 ' '$17.45 ' '$18.65 '
 '$45.98 ' '$49.69 ' '$34.36 ' '$55.18 ' '$52.88 ' '$61.16 ' '$82.83 '
 '$86.67 ' '$61.17 ' '$128.76 ' '$73.11 ' '$101.97 ' '$160.93 ' '$229.93 '
 '$152.94 ' '$960.82 ' '$392.60 ' '$216.12 ' '$505.85 ' '$527.53 '
 '$389.26 ' '$54.72 ' '$35.18 ' '$59.32 ' '$50.13 ' '$45.53 ' '$55.99 '
 '$45.83 ' '$43.04 ' '$53.76 ' '$58.36 ' '$29.01 ' '$33.65 ' '$275.46 '
 '$321.44 ' '$152.44 ' '$293.85 ' '$137.60 ' '$152.90 ' '$261.66 '
 '$252.47 ' '$316.85 ' 

In [9]:
# Filter values to keep only digits, dots, and dollar signs
df_product['filtered_values_unit_cost_usd'] = df_product['unit_cost_usd'].str.replace(r'[^\d.$]', '', regex=True)
df_product['filtered_values_unit_price_usd'] = df_product['unit_price_usd'].str.replace(r'[^\d.$]', '', regex=True)
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   productkey                      2517 non-null   int64 
 1   product_name                    2517 non-null   object
 2   brand                           2517 non-null   object
 3   color                           2517 non-null   object
 4   unit_cost_usd                   2517 non-null   object
 5   unit_price_usd                  2517 non-null   object
 6   subcategorykey                  2517 non-null   int64 
 7   subcategory                     2517 non-null   object
 8   categorykey                     2517 non-null   int64 
 9   category                        2517 non-null   object
 10  filtered_values_unit_cost_usd   2517 non-null   object
 11  filtered_values_unit_price_usd  2517 non-null   object
dtypes: int64(3), object(9)
memory usage: 236.1+ KB


In [10]:
# Load and preprocess the Products data 
df_product['unit_cost_usd'] = df_product['unit_cost_usd'].apply(extract_numeric)
df_product['unit_price_usd'] = df_product['unit_price_usd'].apply(extract_numeric)
df_product = df_product.drop(['filtered_values_unit_cost_usd', 'filtered_values_unit_price_usd'], axis=1)
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   productkey      2517 non-null   int64  
 1   product_name    2517 non-null   object 
 2   brand           2517 non-null   object 
 3   color           2517 non-null   object 
 4   unit_cost_usd   2517 non-null   float64
 5   unit_price_usd  2517 non-null   float64
 6   subcategorykey  2517 non-null   int64  
 7   subcategory     2517 non-null   object 
 8   categorykey     2517 non-null   int64  
 9   category        2517 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 196.8+ KB


In [11]:
for i in df_sales.columns:
    print(f"{i}: count -> {df_sales[i].nunique()}, percentage ->{(df_sales[i].nunique()/len(df_sales))*100}%, null percentage ->{(df_sales[i].isnull().sum()/len(df_sales))*100}%")
print(df_sales.info())
df_sales.head()

order_number: count -> 26326, percentage ->41.86438521722537%, null percentage ->0.0%
line_item: count -> 7, percentage ->0.011131607404109154%, null percentage ->0.0%
order_date: count -> 1641, percentage ->2.6095668214490173%, null percentage ->0.0%
delivery_date: count -> 1492, percentage ->2.3726226067044083%, null percentage ->79.064626932129%
customerkey: count -> 11887, percentage ->18.9030596018065%, null percentage ->0.0%
storekey: count -> 58, percentage ->0.09223331849119012%, null percentage ->0.0%
productkey: count -> 2492, percentage ->3.9628522358628584%, null percentage ->0.0%
quantity: count -> 10, percentage ->0.015902296291584506%, null percentage ->0.0%
currency_code: count -> 5, percentage ->0.007951148145792253%, null percentage ->0.0%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_number   62884 non-null  in

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD


In [12]:
pd.set_option('display.max_rows', None)
df_sales_not_null_avg = df_sales[df_sales['delivery_date'].notnull()][['order_date','delivery_date']]
for i in df_sales_not_null_avg.columns:
    df_sales_not_null_avg[i] = pd.to_datetime(df_sales_not_null_avg[i], errors='coerce')
lag_days = (df_sales_not_null_avg['delivery_date'] - df_sales_not_null_avg['order_date']).dt.days.median()

df_sales['order_date'] = pd.to_datetime(df_sales['order_date'], errors='coerce')

# Impute missing delivery dates
df_sales['delivery_date'].fillna(df_sales['order_date'] + pd.Timedelta(days=lag_days), inplace=True)

df_sales['delivery_date'] = pd.to_datetime(df_sales['delivery_date'], errors='coerce')
print(df_sales.head())
print(df_sales.info())

   order_number  line_item order_date delivery_date  customerkey  storekey  \
0        366000          1 2016-01-01    2016-01-05       265598        10   
1        366001          1 2016-01-01    2016-01-13      1269051         0   
2        366001          2 2016-01-01    2016-01-13      1269051         0   
3        366002          1 2016-01-01    2016-01-12       266019         0   
4        366002          2 2016-01-01    2016-01-12       266019         0   

   productkey  quantity currency_code  
0        1304         1           CAD  
1        1048         2           USD  
2        2007         1           USD  
3        1106         7           CAD  
4         373         1           CAD  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_number   62884 non-null  int64         
 1   line_item      62884 non-

In [13]:
df_sales['currency_code'].unique()

array(['CAD', 'USD', 'GBP', 'EUR', 'AUD'], dtype=object)

In [14]:
for i in df_stores.columns:
    print(f"{i}: count -> {df_stores[i].nunique()}, percentage ->{(df_stores[i].nunique()/len(df_stores))*100}%, null percentage ->{(df_stores[i].isnull().sum()/len(df_stores))*100}%")
print(df_stores.info())
df_stores.head()

storekey: count -> 67, percentage ->100.0%, null percentage ->0.0%
country: count -> 9, percentage ->13.432835820895523%, null percentage ->0.0%
state: count -> 67, percentage ->100.0%, null percentage ->0.0%
square_meters: count -> 36, percentage ->53.73134328358209%, null percentage ->1.4925373134328357%
open_date: count -> 25, percentage ->37.3134328358209%, null percentage ->0.0%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   storekey       67 non-null     int64  
 1   country        67 non-null     object 
 2   state          67 non-null     object 
 3   square_meters  66 non-null     float64
 4   open_date      67 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ KB
None


,storekey,country,state,square_meters,open_date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015


In [15]:
df_stores[df_stores['square_meters'].isnull()]

,storekey,country,state,square_meters,open_date
66,0,Online,Online,NaN,1/1/2010


In [16]:
df_stores['open_date'] = pd.to_datetime(df_stores['open_date'], errors='coerce')
df_stores['square_meters'] = df_stores['square_meters'].fillna(0.0)

In [17]:
df_stores['country'].unique()

array(['Australia', 'Canada', 'France', 'Germany', 'Italy', 'Netherlands',
       'United Kingdom', 'United States', 'Online'], dtype=object)

In [18]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   storekey       67 non-null     int64         
 1   country        67 non-null     object        
 2   state          67 non-null     object        
 3   square_meters  67 non-null     float64       
 4   open_date      67 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 2.7+ KB


In [19]:
for i in df_exchange_rates.columns:
    # if df_exchange_rates[i].dtype == "object" and i not in ['unit_cost_usd','unit_price_usd']:
    #     df_exchange_rates[i] = df_exchange_rates[i].apply(lowercase)
    print(f"{i}: count -> {df_exchange_rates[i].nunique()}, percentage ->{(df_exchange_rates[i].nunique()/len(df_exchange_rates))*100}%, null percentage ->{(df_exchange_rates[i].isnull().sum()/len(df_exchange_rates))*100}%")
print(df_exchange_rates.info())
df_exchange_rates.head()

date: count -> 2243, percentage ->20.0%, null percentage ->0.0%
currency: count -> 5, percentage ->0.04458314757021846%, null percentage ->0.0%
exchange: count -> 3473, percentage ->30.96745430227374%, null percentage ->0.0%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      11215 non-null  object 
 1   currency  11215 non-null  object 
 2   exchange  11215 non-null  float64
dtypes: float64(1), object(2)
memory usage: 263.0+ KB
None


,date,currency,exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415


In [20]:
df_exchange_rates['currency'].unique()

array(['USD', 'CAD', 'AUD', 'EUR', 'GBP'], dtype=object)

In [21]:
df_exchange_rates['date'] = pd.to_datetime(df_exchange_rates['date'], errors='coerce')

In [22]:
df_exchange_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      11215 non-null  datetime64[ns]
 1   currency  11215 non-null  object        
 2   exchange  11215 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 263.0+ KB


In [23]:
# Merge dataframes
merged_df = df_sales.merge(df_customer, on='customerkey', how='left')
merged_df = merged_df.merge(df_stores, on='storekey', how='left', suffixes=('', '_store'))
merged_df = merged_df.merge(df_product, on='productkey', how='left', suffixes=('', '_product'))
merged_df = merged_df.merge(df_exchange_rates, left_on=['order_date', 'currency_code'], right_on=['date', 'currency'], how='left')
merged_df = merged_df.apply(lambda x: x.str.upper() if x.dtype == "object" else x)

In [25]:
merged_df.dtypes

order_number               int64
line_item                  int64
order_date        datetime64[ns]
delivery_date     datetime64[ns]
customerkey                int64
storekey                   int64
productkey                 int64
quantity                   int64
currency_code             object
gender                    object
name                      object
city                      object
state_code                object
state                     object
zip_code                  object
country                   object
continent                 object
birthday          datetime64[ns]
age                        int64
country_store             object
state_store               object
square_meters            float64
open_date         datetime64[ns]
product_name              object
brand                     object
color                     object
unit_cost_usd            float64
unit_price_usd           float64
subcategorykey             int64
subcategory               object
categoryke

In [26]:
import pymysql

# Connection to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='Krisrak@123')

try:
    # Create the database
    with myconnection.cursor() as cursor:
        cursor.execute("CREATE DATABASE IF NOT EXISTS Dataspark_Global_Electronics")
    
    # Loop through each dataframe and create tables
    for df, table_name in zip(
        [df_customer, df_stores, df_product, df_exchange_rates, df_sales, merged_df],
        ['Customers', 'Stores', 'Products', 'Exchange_Rates', 'Sales', 'Merged_df']
    ):
        # Convert columns to uppercase if they are of object type
        df = df.apply(lambda x: x.str.upper() if x.dtype == "object" else x)
        
        # Get column details
        a = ",".join(f"{i} {j}" for i, j in zip(df.columns, df.dtypes))
        a = a.replace("int64", "INT").replace("object", "TEXT").replace("float64", "FLOAT").replace("datetime64[ns]", "DATETIME")
        
        # Create table
        with myconnection.cursor() as cursor:
            cursor.execute(f"CREATE TABLE IF NOT EXISTS Dataspark_Global_Electronics.{table_name} ({a})")
        
        # Insert rows
        sql = f"INSERT INTO Dataspark_Global_Electronics.{table_name} VALUES "
        for j in range(len(df)):
            with myconnection.cursor() as cursor:
                cursor.execute(f"{sql} {tuple(df.iloc[j])}")
            myconnection.commit()
finally:
    myconnection.close()

In [27]:
file_names = {
    'df_customer': 'customer_data.csv',
    'df_stores': 'stores_data.csv',
    'df_product': 'product_data.csv',
    'df_exchange_rates': 'exchange_rates_data.csv',
    'df_sales': 'sales_data.csv',
    'merged_df': 'All_data.csv'
}

df_customer.to_csv(file_names['df_customer'], index=False)
df_stores.to_csv(file_names['df_stores'], index=False)
df_product.to_csv(file_names['df_product'], index=False)
df_exchange_rates.to_csv(file_names['df_exchange_rates'], index=False)
df_sales.to_csv(file_names['df_sales'], index=False)
merged_df.to_csv(file_names['merged_df'], index=False)